In [1]:
#instalo/actualizo
#! pip3 install --upgrade google-cloud-aiplatform


In [7]:
import re
import pandas as pd
from google.cloud import bigquery
#import vertexai 
#from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part, Content
#import vertexai.preview.generative_models as generative_models 
import sys
import os
from fuzzywuzzy import process, fuzz

In [2]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\flori\\OneDrive\\Desktop\\LDP\\ldp-data-genai-producto-b871ebffc681.json"

In [3]:
# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project='ldp-data-wyscout')


PARA EMPEZAR ME TRAIGO LAS TABLAS DE TRANSFERENCIAS DE TFMKT Y WYSCOUT

In [4]:


# Consulta SQL
query2 = """SELECT * FROM ldp-data-wyscout.transfer_market.players_transfers_curated"""

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame
    df1 = client.query(query2).to_dataframe()
    print(df1.head())
except Exception as e:
    print(f"An error occurred: {e}")


  row_ind           player_name season transfer_date         team_from  \
0      11       Ceyhun Gülselam  05/06    2005-07-01     U'haching U19   
1      23  Kevin-Prince Boateng  05/06    2005-07-01     Hertha BSC II   
2       7          Hong-Lim Lee  03/04    2003-07-01      Tai Po Youth   
3      15        Yoel Rodríguez  06/07    2006-07-01         Celta U19   
4      11           Wai-Man Lee  91/92    1991-07-01  Instant-Dict Yth   

           team_to market_value transfer_value contract_expiry  \
0  Unterhaching II           NA             NA              NA   
1       Hertha BSC       250000             NA      2006-06-30   
2           Tai Po           NA             NA              NA   
3     Celta Vigo B           NA             NA              NA   
4          Eastern           NA              0              NA   

                                          player_url  query_date  \
0  https://www.transfermarkt.com/ceyhun-gulselam/...  2024-09-07   
1  https://www.transfe

In [6]:
#df1=pd.read_csv('tfmkt.csv')

In [5]:
df1.head(20)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id
0,11,Ceyhun Gülselam,05/06,2005-07-01,U'haching U19,Unterhaching II,NA,NA,NA,https://www.transfermarkt.com/ceyhun-gulselam/...,2024-09-07,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,37216,https://www.transfermarkt.com/altay-sk/startse...,2375,20050701237537216
1,23,Kevin-Prince Boateng,05/06,2005-07-01,Hertha BSC II,Hertha BSC,250000,NA,2006-06-30,https://www.transfermarkt.com/kevin-prince-boa...,2024-09-10,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,16922,https://www.transfermarkt.com/hertha-bsc/start...,44,200507014416922
2,7,Hong-Lim Lee,03/04,2003-07-01,Tai Po Youth,Tai Po,NA,NA,NA,https://www.transfermarkt.com/hong-lim-lee/pro...,2024-09-11,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,193384,https://www.transfermarkt.com/lee-man/startsei...,61336,2003070161336193384
3,15,Yoel Rodríguez,06/07,2006-07-01,Celta U19,Celta Vigo B,NA,NA,NA,https://www.transfermarkt.com/yoel-rodriguez/p...,2024-09-07,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,58115,https://www.transfermarkt.com/sd-eibar/startse...,1533,20060701153358115
4,11,Wai-Man Lee,91/92,1991-07-01,Instant-Dict Yth,Eastern,NA,0,NA,https://www.transfermarkt.com/wai-man-lee/prof...,2024-09-11,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,362757,https://www.transfermarkt.com/wing-go/startsei...,86237,1991070186237362757
5,23,Fausto Lourenço,02/03,2002-07-01,Académica U15,Padroense U17,NA,NA,NA,https://www.transfermarkt.com/fausto-lourenco/...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,86750,https://www.transfermarkt.com/anadia-fc/starts...,4747,20020701474786750
6,4,Thomas Militão,05/06,2005-07-01,Caldas Youth,Caldas U15,NA,NA,NA,https://www.transfermarkt.com/thomas-militao/p...,2024-09-08,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,556650,https://www.transfermarkt.com/caldas-sc/starts...,4275,200507014275556650
7,9,Christos Karelas,05/06,2005-07-01,TSG Lindau-Zech,FC Wangen 05,NA,NA,NA,https://www.transfermarkt.com/christos-karelas...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,165163,https://www.transfermarkt.com/fc-hochst/starts...,8323,200507018323165163
8,8,Ivan Rakitic,04/05,2004-07-01,FC Basel U18,FC Basel U21,NA,NA,NA,https://www.transfermarkt.com/ivan-rakitic/pro...,2024-09-08,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,32467,https://www.transfermarkt.com/fc-sevilla/start...,368,2004070136832467
9,13,Siu-Yuen Chan,04/05,2004-07-01,Rangers Youth,Rangers,NA,NA,NA,https://www.transfermarkt.com/siu-yuen-chan/pr...,2024-09-08,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,201892,https://www.transfermarkt.com/sai-kung/startse...,62532,2004070162532201892


In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1101741 entries, 0 to 1101740
Data columns (total 17 columns):
 #   Column              Non-Null Count    Dtype              
---  ------              --------------    -----              
 0   row_ind             1101741 non-null  object             
 1   player_name         1101741 non-null  object             
 2   season              1101741 non-null  object             
 3   transfer_date       1101741 non-null  object             
 4   team_from           1101741 non-null  object             
 5   team_to             1101741 non-null  object             
 6   market_value        1101741 non-null  object             
 7   transfer_value      1101741 non-null  object             
 8   contract_expiry     1101741 non-null  object             
 9   player_url          1101741 non-null  object             
 10  query_date          1101741 non-null  dbdate             
 11  timestamp           1101741 non-null  datetime64[us, UTC]
 12  

In [9]:
#dftext = df1.to_csv('tfmkt.csv', index=False)

In [9]:
# Contar valores nulos en df1
null_counts_df1 = df1.isnull().sum()
print("Valores nulos en df1:")
print(null_counts_df1)



Valores nulos en df1:
row_ind                   0
player_name               0
season                    0
transfer_date             0
team_from                 0
team_to                   0
market_value              0
transfer_value            0
contract_expiry           0
player_url                0
query_date                0
timestamp                 0
blob                  96968
player_id                 1
teams_url                 1
team_id                   1
player_transfer_id        1
dtype: int64


In [8]:
###FILTRO LA DE DICT TEAMS POR team_type="parent" and category="default" and gender = 'male' Y ADEMAS POR LOS PAISES QUE ESTAN EN LA DE CAPOLOGY PARA ACHICAR REGISTROS

query = """select * from ldp-data-wyscout.wyscout_curated.transfer_con_nombre;
 """

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame                     ###domestic_competition_name
    df2 = client.query(query).to_dataframe()
    print(df2.head())
except Exception as e:
    print(f"An error occurred: {e}")

                  nombre  transferId  fromTeamId     fromTeamName  toTeamId  \
0              Zen Galea     2252155        <NA>                       <NA>   
1         Daniel Radeski     2252179        <NA>                       8380   
2  Hassan Mushtaq Taheri     2252004        7060     Boo FF Nacka      7143   
3          Lucas Poulsen     2251604       34555        Sundby BK      7463   
4     Kasper Sommerstedt     2408653       11173  ÍF Fuglafjørdur     22910   

              toTeamName active   startDate     endDate      type value  \
0        Mgarr United FC      0  2024-03-01        None   Unknown     0   
1  Stirling Macedonia SC      0  2024-03-01        None   Unknown     0   
2               Follo FK      0  2022-09-08        None  Transfer     0   
3           BK Frem 1886      0  2024-02-01  2024-06-30  Transfer     0   
4           Hedensted BK      0  2024-08-27        None  Transfer     0   

  currency announceDate  playerId  
0      EUR                1061951  
1 

In [10]:
# Contar valores nulos en df2
null_counts_df2 = df2.isnull().sum()
print("Valores nulos en df2:")
print(null_counts_df2)

Valores nulos en df2:
nombre              77
transferId           0
fromTeamId      593112
fromTeamName    198586
toTeamId        293553
toTeamName      229523
active               0
startDate         1026
endDate         696105
type                55
value                0
currency          1395
announceDate         0
playerId             0
dtype: int64


In [14]:
# Filtrar las filas donde fromTeamId es nulo
null_fromTeamId_rows = df2[df2['fromTeamId'].isnull()]

# Mostrar las filas con valores nulos en fromTeamId
null_fromTeamId_rows.head(30)

,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,Zen Galea,2252155,<NA>,,<NA>,Mgarr United FC,0,2024-03-01,None,Unknown,0,EUR,,1061951
1,Daniel Radeski,2252179,<NA>,,8380,Stirling Macedonia SC,0,2024-03-01,None,Unknown,0,EUR,,1061960
8,Alfie Cicale,2251606,<NA>,,4792,VPS,0,2024-04-03,None,Free Transfer,0,EUR,,1061848
9,André Nysted,2252033,<NA>,,7143,Follo,0,2024-04-01,None,Free Transfer,0,EUR,,1061912
10,Alex Clarke,2251810,<NA>,,10911,Linfield,0,2024-01-01,None,Free Transfer,0,EUR,,1061884
12,Frede Aasen Larsen,2252084,<NA>,,7208,Viking II,0,2021-08-01,None,Free Transfer,0,EUR,,1061925
13,Markus Sola,2252093,<NA>,,7208,Viking II,0,2023-01-01,None,Free Transfer,0,EUR,,1061930
14,Frioi Borg,2252360,<NA>,,11178,B68 Toftir,0,2024-01-01,None,Free Transfer,0,EUR,,1061988
15,Hákon Atli Aðalsteinsson,2252366,<NA>,,7724,KA Akureyri,0,2023-02-02,2023-04-06,Free Transfer,0,EUR,,1061989
16,Leo Ritvars Dalbins,2252397,<NA>,,75651,Ogre United,0,2023-01-01,None,Free Transfer,0,EUR,,1061995


In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1933826 entries, 0 to 1933825
Data columns (total 14 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   nombre        object
 1   transferId    Int64 
 2   fromTeamId    Int64 
 3   fromTeamName  object
 4   toTeamId      Int64 
 5   toTeamName    object
 6   active        object
 7   startDate     object
 8   endDate       object
 9   type          object
 10  value         object
 11  currency      object
 12  announceDate  object
 13  playerId      Int64 
dtypes: Int64(4), object(10)
memory usage: 213.9+ MB


In [16]:

# Eliminar filas con valores nulos en df2
df2_clean = df2.dropna()

print("\nDataFrame df2_clean:")
print(df2_clean.head())


DataFrame df2_clean:
                 nombre  transferId  fromTeamId                 fromTeamName  \
3         Lucas Poulsen     2251604       34555                    Sundby BK   
6   Herman Oen Paulsrud     2252078        7194            Stabæk Fotball II   
11         Osama Housni     2252129        7354                Nordstrand IF   
32      Ivan Zheleznyak     1382703       31313  FC Olimpik Donetsk Under 19   
33    Bektur Abdyvaliev     1185550       18646            FK Dordoi Bishkek   

    toTeamId                    toTeamName active   startDate     endDate  \
3       7463                  BK Frem 1886      0  2024-02-01  2024-06-30   
6      24654          Grorud Idrettslag II      0  2022-06-29  2024-03-22   
11      7345                     Grorud IL      0  2024-03-05  2024-08-16   
32     66970  FC Inhulets Petrove Under 19      0  2021-07-30  2022-07-01   
33     62586                     FC Ilbirs      0  2021-03-11  2022-12-31   

             type value currency a

In [ ]:
df2_clean.head(20)

In [21]:
df2_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 854637 entries, 3 to 1933825
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   nombre        854637 non-null  object
 1   transferId    854637 non-null  Int64 
 2   fromTeamId    854637 non-null  Int64 
 3   fromTeamName  854637 non-null  object
 4   toTeamId      854637 non-null  Int64 
 5   toTeamName    854637 non-null  object
 6   active        854637 non-null  object
 7   startDate     854637 non-null  object
 8   endDate       854637 non-null  object
 9   type          854637 non-null  object
 10  value         854637 non-null  object
 11  currency      854637 non-null  object
 12  announceDate  854637 non-null  object
 13  playerId      854637 non-null  Int64 
dtypes: Int64(4), object(10)
memory usage: 101.1+ MB


In [23]:
# Ordenar df1 alfabéticamente por la columna 'player_name'
df1_sorted = df1.sort_values(by='player_name')
print("df1 ordenado alfabéticamente por 'player_name':")




df1 ordenado alfabéticamente por 'player_name':


In [ ]:
df1_sorted.head(50)

In [25]:
# Ordenar df2 alfabéticamente por la columna 'nombre'
df2_sorted = df2_clean.sort_values(by='nombre')
print("\ndf2 ordenado alfabéticamente por 'nombre':")



df2 ordenado alfabéticamente por 'nombre':


In [ ]:
df2_sorted.head(50)

In [11]:
#df2=pd.read_csv('wyscout.csv')

In [12]:
# Reemplazar NaN con un valor por defecto, por ejemplo, -1 o 0
#df2['fromTeamId'] = df2['fromTeamId'].fillna(0).astype(int)
#df2['toTeamId'] = df2['toTeamId'].fillna(0).astype(int)


In [13]:
df2.head()

,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,Herman Oen Paulsrud,2252070,24654,Grorud Idrettslag II,7345,Grorud IL,0,2024-03-22,None,Transfer,0,EUR,,1061921
1,Lucas Poulsen,2352906,7463,BK Frem 1886,34555,Sundby BK,0,2024-08-01,None,Transfer,0,EUR,,1061847
2,Osama Housni,2252131,7292,Holmlia SK,7354,Nordstrand IF,0,2023-07-28,2024-03-05,Transfer,0,EUR,,1061932
3,Ahmad Shaban,2376923,13294,Hapoel Acre FC,13364,Hapoel Afula FC,0,2024-07-01,None,Transfer,0,EUR,,1061976
4,Ryan Pereira Costa,2352994,23709,Os Belenenses SAD Under 19,<NA>,None,0,2024-07-30,None,Free Agent,0,EUR,,1061882


In [14]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1933826 entries, 0 to 1933825
Data columns (total 14 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   nombre        object
 1   transferId    Int64 
 2   fromTeamId    Int64 
 3   fromTeamName  object
 4   toTeamId      Int64 
 5   toTeamName    object
 6   active        object
 7   startDate     object
 8   endDate       object
 9   type          object
 10  value         object
 11  currency      object
 12  announceDate  object
 13  playerId      Int64 
dtypes: Int64(4), object(10)
memory usage: 213.9+ MB


In [15]:
#dftext2 = df2.to_csv('wyscout.csv',index=False)

In [32]:
# Tomar las primeras 5000 filas de df1_sorted y df2_sorted
df1_sorted_top5000 = df1_sorted.head(5000)
df2_sorted_top5000 = df2_sorted.head(5000)

# Mostrar las primeras filas para verificar
print(df1_sorted_top5000.head())
print(df2_sorted_top5000.head())

        row_ind          player_name season transfer_date       team_from  \
1049718       1            - Stewart  23/24    2023-07-01  Gala Fairydean   
579292        1               . Bana  20/21    2020-07-01       Own Youth   
981330        2  A Mohamed Liyaakath  23/24    2023-06-01   Chennaiyin FC   
528085        4  A Mohamed Liyaakath  20/21    2020-11-01  Chennaiyin U16   
966534        1  A Mohamed Liyaakath  23/24    2024-02-13    Without Club   

               team_to market_value transfer_value contract_expiry  \
1049718   Without Club           NA             NA              NA   
579292     Desp. Huíla           NA             NA              NA   
981330    Without Club        25000             NA              NA   
528085   Indian Arrows           NA              0              NA   
966534     G. Kerala B        25000             NA              NA   

                                                player_url  query_date  \
1049718  https://www.transfermarkt.com/-st

In [31]:
import pandas as pd
import unicodedata
from fuzzywuzzy import fuzz

In [ ]:


# Función para normalizar y simplificar nombres de jugadores y equipos
def normalize_text(text):
    if isinstance(text, str):
        # Convertir a minúsculas, normalizar acentos y eliminar exceso de espacios
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        text = ' '.join(text.lower().strip().split())
        return text
    return text

def simplify_name(name):
    parts = name.split()
    # Retorna sólo el primer y último nombre
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    # Convierte a minúsculas y elimina palabras comunes (ajusta según tus necesidades)
    ignore_words = {'fc', 'club', 'de', 'the'}
    parts = club_name.lower().split()
    return ' '.join([word for word in parts if word not in ignore_words])

# Cargar los archivos CSV en dataframes
#df1_sorted_top5000 = pd.read_csv('archivo1.csv')
#df2_sorted_top5000 = pd.read_csv('archivo2.csv')

# Normalizar y simplificar columnas relevantes
df1_sorted_top5000['player_name'] = df1_sorted_top5000['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted_top5000['team_from'] = df1_sorted_top5000['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top5000['team_to'] = df1_sorted_top5000['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top5000['transfer_date'] = pd.to_datetime(df1_sorted_top5000['transfer_date'], errors='coerce')

df2_sorted_top5000['nombre'] = df2_sorted_top5000['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted_top5000['fromTeamName'] = df2_sorted_top5000['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top5000['toTeamName'] = df2_sorted_top5000['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top5000['startDate'] = pd.to_datetime(df2_sorted_top5000['startDate'], errors='coerce')

# Crear un ID temporal basado en mes y año
df1_sorted_top5000['ID'] = df1_sorted_top5000.apply(lambda row: f"{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted_top5000['ID'] = df2_sorted_top5000.apply(lambda row: f"{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Unión basada en IDs aproximados y similitud de nombres
merged_rows = []

for idx1, row1 in df1_sorted_top5000.iterrows():
    for idx2, row2 in df2_sorted_top5000.iterrows():
        if row1['ID'] == row2['ID']:
            # Usar similitud fuzzywuzzy para nombre del jugador
            name_similarity = fuzz.partial_ratio(row1['player_name'], row2['nombre'])
            # Verifica que la similitud supere un umbral (ajustable según necesidad)
            if name_similarity > 80:
                merged_rows.append(row1.to_dict() | row2.to_dict())

# Crear un dataframe con los resultados de la unión
merged_df2 = pd.DataFrame(merged_rows)

# Ver los resultados
print(merged_df2.head())

# Guardar a un archivo CSV
merged_df2.to_csv('archivo_unido2.csv', index=False)


In [36]:


# Funciones auxiliares para la normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    parts = name.split()
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    ignore_words = {'fc', 'club', 'de', 'the'}
    return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])

# Normalizar y simplificar columnas relevantes
df1_sorted_top5000['player_name'] = df1_sorted_top5000['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted_top5000['team_from'] = df1_sorted_top5000['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top5000['team_to'] = df1_sorted_top5000['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top5000['transfer_date'] = pd.to_datetime(df1_sorted_top5000['transfer_date'], errors='coerce')

df2_sorted_top5000['nombre'] = df2_sorted_top5000['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted_top5000['fromTeamName'] = df2_sorted_top5000['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top5000['toTeamName'] = df2_sorted_top5000['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top5000['startDate'] = pd.to_datetime(df2_sorted_top5000['startDate'], errors='coerce')

# Crear un ID temporal basado en mes y año
df1_sorted_top5000['ID'] = df1_sorted_top5000.apply(lambda row: f"{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted_top5000['ID'] = df2_sorted_top5000.apply(lambda row: f"{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Primaria unión por ID
initial_merge = pd.merge(df1_sorted_top5000, df2_sorted_top5000, on='ID', suffixes=('_df1', '_df2'))

# Unión final basada en similitud
merged_rows = []

for _, merged_row in initial_merge.iterrows():
    name_similarity = fuzz.partial_ratio(merged_row['player_name'], merged_row['nombre'])
    if name_similarity > 80:
        merged_rows.append(merged_row)

# Crear un dataframe final con los resultados
merged_df = pd.DataFrame(merged_rows)

# Ver los resultados
print(merged_df.head())

# Guardar el resultado en un archivo CSV
merged_df.to_csv('archivo_unido.csv', index=False)


  row_ind           player_name season transfer_date       team_from  \
0       3            aj cochran  20/21    2021-02-17  phoenix rising   
1       2         aaren d'silva  20/21    2020-10-28          goa ii   
2       5  aaron amadi-holloway  21/22    2021-07-23  sc east bengal   
3       2  aaron amadi-holloway  22/23    2022-08-05   burton albion   
4      16  aaron amadi-holloway  11/12    2011-11-24    bristol city   

         team_to market_value transfer_value contract_expiry  \
0    indy eleven       250000              0              NA   
1            goa        75000             NA              NA   
2  burton albion        2e+05              0              NA   
3       hereford        2e+05              0              NA   
4      bath city        50000             NA      2012-06-30   

                                          player_url  ... toTeamId  \
0  https://www.transfermarkt.com/aj-cochran/profi...  ...    27170   
1  https://www.transfermarkt.com/aaren-dsi

C:\Users\flori\AppData\Local\Temp\ipykernel_25392\1654267820.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top5000['player_name'] = df1_sorted_top5000['player_name'].apply(normalize_text).apply(simplify_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_25392\1654267820.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top5000['team_from'] = df1_sorted_top5000['team_from'].apply(normalize_text).apply(simplify_club_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_25392\1654267

In [39]:

pd.set_option('display.max_columns', None)

In [40]:
merged_df.head(20)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id,ID,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,3,aj cochran,20/21,2021-02-17,phoenix rising,indy eleven,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202102174115306391,phoenix rising_indy eleven_2021_2,aj cochran,1156285,29992,phoenix rising,27170,indy eleven,0,2021-02-17,2023-02-02,Transfer,0,EUR,,300553
1,2,aaren d'silva,20/21,2020-10-28,goa ii,goa,75000,NA,NA,https://www.transfermarkt.com/aaren-dsilva/pro...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,630382,https://www.transfermarkt.com/hyderabad-fc/sta...,76364,2020102876364630382,goa ii_goa_2020_10,aaren d'silva,1059236,61377,goa ii,31884,goa,0,2020-10-28,2021-06-18,Transfer,0,EUR,,703579
2,5,aaron amadi-holloway,21/22,2021-07-23,sc east bengal,burton albion,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2021072346842186574,sc east bengal_burton albion_2021_7,aaron amadi-holloway,1350346,15036,sc east bengal,1710,burton albion,0,2021-07-23,2022-01-28,Transfer,0,EUR,,10612
3,2,aaron amadi-holloway,22/23,2022-08-05,burton albion,hereford,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2022080546842186574,burton albion_hereford_2022_8,aaron amadi-holloway,1674586,1710,burton albion,36605,hereford,0,2022-08-05,2023-07-12,Transfer,0,EUR,,10612
4,16,aaron amadi-holloway,11/12,2011-11-24,bristol city,bath city,50000,NA,2012-06-30,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2011112446842186574,bristol city_bath city_2011_11,aaron amadi-holloway,80644,1655,bristol city,1728,bath city,0,2011-11-24,2012-01-01,Loan,0,,2011-11-26,10612
5,6,aaron amadi-holloway,20/21,2020-10-17,brisbane roar,sc east bengal,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2020101746842186574,brisbane roar_sc east bengal_2020_10,aaron amadi-holloway,1054694,8314,brisbane roar,15036,sc east bengal,0,2020-10-17,2021-07-23,Free Transfer,0,EUR,,10612
6,15,aaron amadi-holloway,11/12,2012-01-01,bath city,bristol city,50000,NA,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2012010146842186574,bath city_bristol city_2012_1,aaron amadi-holloway,1572374,1728,bath city,1655,bristol city,0,2012-01-01,2014-02-12,Back from Loan,0,EUR,,10612
7,6,aaron arnott,22/23,2023-01-13,brechin city,raith rovers,125000,NA,2023-05-31,https://www.transfermarkt.com/aaron-arnott/pro...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,822785,https://www.transfermarkt.com/brechin-city-fc/...,3194,202301133194822785,brechin city_raith rovers_2023_1,aaron arnott,1771045,8529,brechin city,8527,raith rovers,0,2023-01-13,2023-01-20,Transfer,0,EUR,,699833
8,3,aaron arnott,23/24,2023-09-19,raith rovers,east kilbride,125000,NA,2024-05-31,https://www.transfermarkt.com/aaron-arnott/pro...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_tr

In [41]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 231 entries, 0 to 250
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   row_ind             231 non-null    object             
 1   player_name         231 non-null    object             
 2   season              231 non-null    object             
 3   transfer_date       231 non-null    datetime64[ns]     
 4   team_from           231 non-null    object             
 5   team_to             231 non-null    object             
 6   market_value        231 non-null    object             
 7   transfer_value      231 non-null    object             
 8   contract_expiry     231 non-null    object             
 9   player_url          231 non-null    object             
 10  query_date          231 non-null    object             
 11  timestamp           231 non-null    datetime64[ns, UTC]
 12  blob                218 non-null    objec

In [42]:
# Funciones auxiliares para la normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    parts = name.split()
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    ignore_words = {'fc', 'club', 'de', 'the'}
    return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])

# Normalizar y simplificar columnas relevantes
df1_sorted['player_name'] = df1_sorted['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted['team_from'] = df1_sorted['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['team_to'] = df1_sorted['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['transfer_date'] = pd.to_datetime(df1_sorted['transfer_date'], errors='coerce')

df2_sorted['nombre'] = df2_sorted['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted['fromTeamName'] = df2_sorted['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['toTeamName'] = df2_sorted['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['startDate'] = pd.to_datetime(df2_sorted['startDate'], errors='coerce')

# Crear un ID temporal basado en mes y año
df1_sorted['ID'] = df1_sorted.apply(lambda row: f"{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted['ID'] = df2_sorted.apply(lambda row: f"{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Primaria unión por ID
initial_merge = pd.merge(df1_sorted, df2_sorted, on='ID', suffixes=('_df1', '_df2'))

# Unión final basada en similitud
merged_rows = []

for _, merged_row in initial_merge.iterrows():
    name_similarity = fuzz.partial_ratio(merged_row['player_name'], merged_row['nombre'])
    if name_similarity > 80:
        merged_rows.append(merged_row)

# Crear un dataframe final con los resultados
merged_df3 = pd.DataFrame(merged_rows)

# Ver los resultados
print(merged_df3.head())

# Guardar el resultado en un archivo CSV
merged_df3.to_csv('archivo_unido3.csv', index=False)

  row_ind           player_name season transfer_date       team_from  \
0       3            aj cochran  20/21    2021-02-17  phoenix rising   
1       2        aakash sangwan  22/23    2022-06-21          punjab   
2       2         aaren d'silva  20/21    2020-10-28          goa ii   
4       5  aaron amadi-holloway  21/22    2021-07-23  sc east bengal   
5       2  aaron amadi-holloway  22/23    2022-08-05   burton albion   

         team_to market_value transfer_value contract_expiry  \
0    indy eleven       250000              0              NA   
1     chennaiyin       125000              0              NA   
2            goa        75000             NA              NA   
4  burton albion        2e+05              0              NA   
5       hereford        2e+05              0              NA   

                                          player_url  query_date  \
0  https://www.transfermarkt.com/aj-cochran/profi...  2024-09-10   
1  https://www.transfermarkt.com/aakash-sangwa

In [43]:
merged_df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34063 entries, 0 to 58243
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   row_ind             34063 non-null  object             
 1   player_name         34063 non-null  object             
 2   season              34063 non-null  object             
 3   transfer_date       34063 non-null  datetime64[ns]     
 4   team_from           34063 non-null  object             
 5   team_to             34063 non-null  object             
 6   market_value        34063 non-null  object             
 7   transfer_value      34063 non-null  object             
 8   contract_expiry     34063 non-null  object             
 9   player_url          34063 non-null  object             
 10  query_date          34063 non-null  object             
 11  timestamp           34063 non-null  datetime64[ns, UTC]
 12  blob                31249 non-null  o

In [49]:
merged_df3.head(20)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id,ID,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,3,aj cochran,20/21,2021-02-17,phoenix rising,indy eleven,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202102174115306391,phoenix rising_indy eleven_2021_2,aj cochran,1156285,29992,phoenix rising,27170,indy eleven,0,2021-02-17,2023-02-02,Transfer,0,EUR,,300553
1,2,aakash sangwan,22/23,2022-06-21,punjab,chennaiyin,125000,0,NA,https://www.transfermarkt.com/aakash-sangwan/p...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,503167,https://www.transfermarkt.com/chennaiyin-fc/st...,45299,2022062145299503167,punjab_chennaiyin_2022_6,akash sangwan,1649519,37206,punjab,31882,chennaiyin,0,2022-06-21,2024-06-12,Transfer,0,EUR,,503194
2,2,aaren d'silva,20/21,2020-10-28,goa ii,goa,75000,NA,NA,https://www.transfermarkt.com/aaren-dsilva/pro...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,630382,https://www.transfermarkt.com/hyderabad-fc/sta...,76364,2020102876364630382,goa ii_goa_2020_10,aaren d'silva,1059236,61377,goa ii,31884,goa,0,2020-10-28,2021-06-18,Transfer,0,EUR,,703579
4,5,aaron amadi-holloway,21/22,2021-07-23,sc east bengal,burton albion,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2021072346842186574,sc east bengal_burton albion_2021_7,aaron amadi-holloway,1350346,15036,sc east bengal,1710,burton albion,0,2021-07-23,2022-01-28,Transfer,0,EUR,,10612
5,2,aaron amadi-holloway,22/23,2022-08-05,burton albion,hereford,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2022080546842186574,burton albion_hereford_2022_8,aaron amadi-holloway,1674586,1710,burton albion,36605,hereford,0,2022-08-05,2023-07-12,Transfer,0,EUR,,10612
6,16,aaron amadi-holloway,11/12,2011-11-24,bristol city,bath city,50000,NA,2012-06-30,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2011112446842186574,bristol city_bath city_2011_11,aaron amadi-holloway,80644,1655,bristol city,1728,bath city,0,2011-11-24,2012-01-01,Loan,0,,2011-11-26,10612
8,6,aaron amadi-holloway,20/21,2020-10-17,brisbane roar,sc east bengal,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2020101746842186574,brisbane roar_sc east bengal_2020_10,aaron amadi-holloway,1054694,8314,brisbane roar,15036,sc east bengal,0,2020-10-17,2021-07-23,Free Transfer,0,EUR,,10612
10,15,aaron amadi-holloway,11/12,2012-01-01,bath city,bristol city,50000,NA,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2012010146842186574,bath city_bristol city_2012_1,aaron amadi-holloway,1572374,1728,bath city,1655,bristol city,0,2012-01-01,2014-02-12,Back from Loan,0,EUR,,10612
13,6,aaron arnott,22/23,2023-01-13,brechin city,raith rovers,125000,NA,2023-05-31,https://www.transfermarkt.com/aaron-arnott/pro...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf..

In [ ]:
merger

In [46]:
import pandas as pd
import unicodedata
from fuzzywuzzy import fuzz

# Funciones auxiliares para la normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    parts = name.split()
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    ignore_words = {'fc', 'club', 'de', 'the'}
    return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])

# Normalizar y simplificar columnas relevantes
df1_sorted['player_name'] = df1_sorted['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted['team_from'] = df1_sorted['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['team_to'] = df1_sorted['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['transfer_date'] = pd.to_datetime(df1_sorted['transfer_date'], errors='coerce')

df2_sorted['nombre'] = df2_sorted['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted['fromTeamName'] = df2_sorted['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['toTeamName'] = df2_sorted['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['startDate'] = pd.to_datetime(df2_sorted['startDate'], errors='coerce')

# Crear un ID temporal basado en los nombres incluidos
df1_sorted['ID'] = df1_sorted.apply(lambda row: f"{row['player_name']}_{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted['ID'] = df2_sorted.apply(lambda row: f"{row['nombre']}_{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Primaria unión por ID
initial_merge = pd.merge(df1_sorted, df2_sorted, on='ID', suffixes=('_df1', '_df2'))

# Unión final basada en similitud (ya no es muy necesaria si el ID incluye nombres)
merged_rows = []

for _, merged_row in initial_merge.iterrows():
    name_similarity = fuzz.partial_ratio(merged_row['player_name'], merged_row['nombre'])
    if name_similarity > 80:  # Esta parte podría omitirse si el ID ya diferencia lo suficiente
        merged_rows.append(merged_row)

# Crear un dataframe final con los resultados
merged_df4 = pd.DataFrame(merged_rows)

# Ver los resultados
print(merged_df4.head())

# Guardar el resultado en un archivo CSV
merged_df4.to_csv('archivo_unido4.csv', index=False)


  row_ind           player_name season transfer_date       team_from  \
0       3            aj cochran  20/21    2021-02-17  phoenix rising   
1       2         aaren d'silva  20/21    2020-10-28          goa ii   
2       5  aaron amadi-holloway  21/22    2021-07-23  sc east bengal   
3       2  aaron amadi-holloway  22/23    2022-08-05   burton albion   
4      16  aaron amadi-holloway  11/12    2011-11-24    bristol city   

         team_to market_value transfer_value contract_expiry  \
0    indy eleven       250000              0              NA   
1            goa        75000             NA              NA   
2  burton albion        2e+05              0              NA   
3       hereford        2e+05              0              NA   
4      bath city        50000             NA      2012-06-30   

                                          player_url  query_date  \
0  https://www.transfermarkt.com/aj-cochran/profi...  2024-09-10   
1  https://www.transfermarkt.com/aaren-dsilva/

In [48]:
merged_df4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31075 entries, 0 to 31074
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   row_ind             31075 non-null  object             
 1   player_name         31075 non-null  object             
 2   season              31075 non-null  object             
 3   transfer_date       31075 non-null  datetime64[ns]     
 4   team_from           31075 non-null  object             
 5   team_to             31075 non-null  object             
 6   market_value        31075 non-null  object             
 7   transfer_value      31075 non-null  object             
 8   contract_expiry     31075 non-null  object             
 9   player_url          31075 non-null  object             
 10  query_date          31075 non-null  object             
 11  timestamp           31075 non-null  datetime64[ns, UTC]
 12  blob                29283 non-null  o

In [50]:
merged_df4.head()

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id,ID,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,3,aj cochran,20/21,2021-02-17,phoenix rising,indy eleven,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202102174115306391,aj cochran_phoenix rising_indy eleven_2021_2,aj cochran,1156285,29992,phoenix rising,27170,indy eleven,0,2021-02-17,2023-02-02,Transfer,0,EUR,,300553
1,2,aaren d'silva,20/21,2020-10-28,goa ii,goa,75000,NA,NA,https://www.transfermarkt.com/aaren-dsilva/pro...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,630382,https://www.transfermarkt.com/hyderabad-fc/sta...,76364,2020102876364630382,aaren d'silva_goa ii_goa_2020_10,aaren d'silva,1059236,61377,goa ii,31884,goa,0,2020-10-28,2021-06-18,Transfer,0,EUR,,703579
2,5,aaron amadi-holloway,21/22,2021-07-23,sc east bengal,burton albion,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2021072346842186574,aaron amadi-holloway_sc east bengal_burton alb...,aaron amadi-holloway,1350346,15036,sc east bengal,1710,burton albion,0,2021-07-23,2022-01-28,Transfer,0,EUR,,10612
3,2,aaron amadi-holloway,22/23,2022-08-05,burton albion,hereford,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2022080546842186574,aaron amadi-holloway_burton albion_hereford_20...,aaron amadi-holloway,1674586,1710,burton albion,36605,hereford,0,2022-08-05,2023-07-12,Transfer,0,EUR,,10612
4,16,aaron amadi-holloway,11/12,2011-11-24,bristol city,bath city,50000,NA,2012-06-30,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2011112446842186574,aaron amadi-holloway_bristol city_bath city_20...,aaron amadi-holloway,80644,1655,bristol city,1728,bath city,0,2011-11-24,2012-01-01,Loan,0,,2011-11-26,10612


DOCU FUZZY  https://pypi.org/project/fuzzywuzzy/

In [53]:
fuzz.ratio("this is a test", "this is a test!")

97

In [54]:
fuzz.partial_ratio("this is a test", "this is a test!")
 

100

In [55]:
fuzz.token_sort_ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

100

In [57]:
fuzz.token_sort_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

84

In [58]:
fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

100